In [2]:
#!pip install sparqlwrapper
#!pip install git+https://github.com/rdflib/sparqlwrapper#egg=sparqlwrapper

In [1]:
import pandas as pd 
import numpy as np

In [2]:
DRKG_PATH = "/Users/elifozkan/Desktop/translator-openpredict/data/drkg/embed/drkg.tsv"
drkg_data = pd.read_table(DRKG_PATH, sep = "\t")
#entity2src = pd.read_table("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/entity2src.tsv",sep = "\t",on_bad_lines='skip')


In [3]:
entity2src = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/entity2src.csv",on_bad_lines='skip')

In [4]:
entity2src.head()

,Subject,Source1,Source2,Source3,Source4,Source5,Source6
0,Gene::1,[Bibliograpic sources] Collection of datasourc...,[Drugbank] https://www.drugbank.ca/atc,[STRING] https://string-db.org/,[Hetionet] Biomedical knowledge graph https://...,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/
1,Gene::1000,[Hetionet] Biomedical knowledge graph https://...,[STRING] https://string-db.org/,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/,NaN,NaN
2,Gene::10000,[STRING] https://string-db.org/,[DGIdb] http://www.dgidb.org/,[Hetionet] Biomedical knowledge graph https://...,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/,NaN
3,Gene::100000711,[GNBR] Data extracted from biomedical texts se...,NaN,NaN,NaN,NaN,NaN
4,Gene::100000800,[GNBR] Data extracted from biomedical texts se...,NaN,NaN,NaN,NaN,NaN


In [5]:
entity2src["Source1"].unique()
BibSource = '[Bibliograpic sources] Collection of datasources from paper: https://www.biorxiv.org/content/10.1101/2020.03.11.986836v1.full.pdf'
BibSource1 = '[Bibliograpic sources] Coronavirus host proteins extracted from https://www.biorxiv.org/content/10.1101/2020.03.22.002386v3?from=groupmessage&isappinstalled=0'
BibSource2 = '[Bibliograpic sources] Extracted from the paper Zhou, Y., Hou, Y., Shen, J. et al. Network-based drug repurposing for novel coronavirus 2019-nCoV/SARS-CoV-2. Cell Discov 6, 14 (2020).'
Hetionet = "[Hetionet] Biomedical knowledge graph https://het.io/about/"
STRING = '[STRING] https://string-db.org/'
DrugBank = '[Drugbank] https://www.drugbank.ca/atc'
GNBR =  '[GNBR] Data extracted from biomedical texts see https://www.ncbi.nlm.nih.gov/pubmed/29490008/'
DGIdb = '[DGIdb] http://www.dgidb.org/'

In [6]:
def split_sources(df):
    split_df = df.copy()
    sources = ["Source1", "Source2", "Source3", "Source4", "Source5", "Source6"]
    for source in sources: 
        split_df[source] = split_df[source].str.split()
    return split_df
        

def get_source_name(split_df):
    source_df = split_df.copy()
    sources = ["Source1", "Source2", "Source3", "Source4", "Source5", "Source6"]
    for source in sources : 
        source_df[source] = source_df[source].str[0]
    return source_df


In [7]:
split_df = split_sources(entity2src)
source_df = get_source_name(split_df)

In [8]:
def split_entity_types(df) : 
    df["SplitSubject"] = df["Subject"].str.split("::")

In [9]:
split_entity_types(source_df)
source_df

,Subject,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject
0,Gene::1,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 1]"
1,Gene::1000,[Hetionet],[STRING],[GNBR],[IntAct],NaN,NaN,"[Gene, 1000]"
2,Gene::10000,[STRING],[DGIdb],[Hetionet],[GNBR],[IntAct],NaN,"[Gene, 10000]"
3,Gene::100000711,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000711]"
4,Gene::100000800,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000800]"
...,...,...,...,...,...,...,...,...
96097,Side Effect::C3665624,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665624]"
96098,Side Effect::C3665770,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665770]"
96099,Side Effect::C3665818,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665818]"
96100,Side Effect::C3665888,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665888]"


In [10]:
def create_entity_type_df(df): 
    entity_type_df = df.copy()
    entity_type_df["SubjectType"] = entity_type_df["SplitSubject"].str[0]
    entity_type_df["ID"] = entity_type_df["SplitSubject"].str[1]
    return entity_type_df

In [11]:
entity_type_df =create_entity_type_df(source_df)
entity_type_df

,Subject,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject,SubjectType,ID
0,Gene::1,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 1]",Gene,1
1,Gene::1000,[Hetionet],[STRING],[GNBR],[IntAct],NaN,NaN,"[Gene, 1000]",Gene,1000
2,Gene::10000,[STRING],[DGIdb],[Hetionet],[GNBR],[IntAct],NaN,"[Gene, 10000]",Gene,10000
3,Gene::100000711,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000711]",Gene,100000711
4,Gene::100000800,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000800]",Gene,100000800
...,...,...,...,...,...,...,...,...,...,...
96097,Side Effect::C3665624,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665624]",Side Effect,C3665624
96098,Side Effect::C3665770,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665770]",Side Effect,C3665770
96099,Side Effect::C3665818,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665818]",Side Effect,C3665818
96100,Side Effect::C3665888,[Hetionet],NaN,NaN,NaN,NaN,NaN,"[Side Effect, C3665888]",Side Effect,C3665888


In [12]:
drkg_data

,Subject,Predicate,Object
0,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2157
1,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::5264
2,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2158
3,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::3309
4,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::28912
...,...,...,...
5874256,Gene::29099,STRING::REACTION::Gene:Gene,Gene::1643
5874257,Gene::51645,STRING::REACTION::Gene:Gene,Gene::3183
5874258,Gene::865,STRING::CATALYSIS::Gene:Gene,Gene::983
5874259,Gene::1066,STRING::BINDING::Gene:Gene,Gene::7365


In [13]:
drkg = pd.merge(drkg_data, entity_type_df, on="Subject")
drkg

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject,SubjectType,ID
0,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::5566,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 6187]",Gene,6187
1,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7412,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 6187]",Gene,6187
2,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3845,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 6187]",Gene,6187
3,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3008,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 6187]",Gene,6187
4,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7083,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 6187]",Gene,6187
...,...,...,...,...,...,...,...,...,...,...,...,...
5343001,Gene::645843,STRING::OTHER::Gene:Gene,Gene::28978,[Hetionet],[STRING],NaN,NaN,NaN,NaN,"[Gene, 645843]",Gene,645843
5343002,Gene::153396,STRING::OTHER::Gene:Gene,Gene::145942,[Hetionet],[STRING],NaN,NaN,NaN,NaN,"[Gene, 153396]",Gene,153396
5343003,Gene::101928147,STRING::OTHER::Gene:Gene,Gene::93661,[STRING],NaN,NaN,NaN,NaN,NaN,"[Gene, 101928147]",Gene,101928147
5343004,Gene::23591,STRING::OTHER::Gene:Gene,Gene::28986,[STRING],NaN,NaN,NaN,NaN,NaN,"[Gene, 23591]",Gene,23591


In [14]:
drkg["Source1"].unique()

array(['[Bibliograpic', '[Drugbank]', '[Hetionet]', '[IntAct]', '[GNBR]',
       '[DGIdb]', '[STRING]'], dtype=object)

In [15]:
Hetionet = drkg.loc[(drkg["Source1"] == "[Hetionet]") | (drkg["Source2"] == "[Hetionet]") | (drkg["Source3"] == "[Hetionet]")
            | (drkg["Source4"] == "[Hetionet]")| (drkg["Source5"] == "[Hetionet]") | (drkg["Source6"] == "[Hetionet]")]

DrugBank = drkg.loc[(drkg["Source1"] == "[Drugbank]") | (drkg["Source2"] == "[Drugbank]") | (drkg["Source3"] == "[Drugbank]")
            | (drkg["Source4"] == "[Drugbank]")| (drkg["Source5"] == "[Drugbank]") | (drkg["Source6"] == "[Drugbank]")]

GNBR = drkg.loc[(drkg["Source1"] == "[GNBR]") | (drkg["Source2"] == "[GNBR]") | (drkg["Source3"] == "[GNBR]")
            | (drkg["Source4"] == "[GNBR]")| (drkg["Source5"] == "[GNBR]") | (drkg["Source6"] == "[GNBR]")]

IntAct = drkg.loc[(drkg["Source1"] == "[IntAct]") | (drkg["Source2"] == "[IntAct]") | (drkg["Source3"] == "[IntAct]")
            | (drkg["Source4"] == "[IntAct]")| (drkg["Source5"] == "[IntAct]") | (drkg["Source6"] == "[IntAct]")]

STRING = drkg.loc[(drkg["Source1"] == "[STRING]") | (drkg["Source2"] == "[STRING]") | (drkg["Source3"] == "[STRING]")
            | (drkg["Source4"] == "[STRING]")| (drkg["Source5"] == "[STRING]") | (drkg["Source6"] == "[STRING]")]

DGIdb = drkg.loc[(drkg["Source1"] == "[DGIdb]") | (drkg["Source2"] == "[DGIdb]") | (drkg["Source3"] == "[DGIdb]")
            | (drkg["Source4"] == "[DGIdb]")| (drkg["Source5"] == "[DGIdb]") | (drkg["Source6"] == "[DGIdb]")]

### Side effect Similarity for Drugs 

>> Excluded entities with only Bibliographic sources

In [16]:
sider_drugs = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/sider_drugnames.csv")
drugbank_drugs = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/drugbank vocabulary.csv")
dgidb_drugs = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/dgidb_drugs.csv")

In [17]:
drugbank_drugs.rename(columns = {"DrugBank ID" : "ID"}, inplace=True)
dgidb_drugs.rename(columns={"concept_id" : "ID"}, inplace=True)


In [18]:
dgidb_drugs = dgidb_drugs[dgidb_drugs['ID'].notna()]
dgidb_drugs["ID"] = dgidb_drugs["ID"].str.replace("chembl:","")


In [19]:
Drugbank_drug_entites = DrugBank.loc[DrugBank["SubjectType"] == "Compound"]
DGIdb_drug_entities = DGIdb.loc[DGIdb["SubjectType"] == "Compound"]
Hetionet_drugs = Hetionet.loc[Hetionet["SubjectType"] == "Compound"]
GNBR_drugs = GNBR.loc[GNBR["SubjectType"] == "Compound"]

In [20]:
Drugbank_drug_entites = pd.merge(drugbank_drugs, Drugbank_drug_entites, on = "ID")
DGIdb_drug_entities_chembl = pd.merge(dgidb_drugs,DGIdb_drug_entities, on = "ID")
DGIdb_drug_entities_db = pd.merge(drugbank_drugs, DGIdb_drug_entities, on = "ID")
Hetionet_drug_entities = pd.merge(drugbank_drugs,Hetionet_drugs, on = "ID")
GNBR_drug_entities = pd.merge(drugbank_drugs, GNBR_drugs, on = "ID")

In [21]:
DGIdb_drug_entities_chembl = DGIdb_drug_entities_chembl.drop(["drug_claim_source","drug_claim_name"],axis=1)

In [22]:
Drugbank_drug_entites = Drugbank_drug_entites.drop(["Accession Numbers", "CAS", "UNII", "Standard InChI Key","Synonyms"],axis=1)
Drugbank_drug_entites.rename(columns={"Common name": "drug_name"},inplace=True)

DGIdb_drug_entities_db = DGIdb_drug_entities_db.drop(["Accession Numbers", "CAS", "UNII", "Standard InChI Key","Synonyms"],axis=1)
DGIdb_drug_entities_db.rename(columns={"Common name": "drug_name"},inplace=True)

Hetionet_drug_entities = Hetionet_drug_entities.drop(["Accession Numbers", "CAS", "UNII", "Standard InChI Key","Synonyms"],axis=1)
Hetionet_drug_entities.rename(columns={"Common name": "drug_name"},inplace=True)

GNBR_drug_entities = GNBR_drug_entities.drop(["Accession Numbers", "CAS", "UNII", "Standard InChI Key","Synonyms"],axis=1)
GNBR_drug_entities.rename(columns={"Common name": "drug_name"},inplace = True)

In [23]:
drugs = [Drugbank_drug_entites, DGIdb_drug_entities_db, DGIdb_drug_entities_chembl,Hetionet_drug_entities, GNBR_drug_entities]
drug_entities = pd.concat(drugs)
drug_entities["drug_name"] = drug_entities["drug_name"].str.lower()
drug_entities

,ID,drug_name,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject,SubjectType
0,DB00001,lepirudin,Compound::DB00001,DRUGBANK::x-atc::Compound:Atc,Atc::B01AE02,[Drugbank],[GNBR],[DGIdb],NaN,NaN,NaN,"[Compound, DB00001]",Compound
1,DB00001,lepirudin,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB06605,[Drugbank],[GNBR],[DGIdb],NaN,NaN,NaN,"[Compound, DB00001]",Compound
2,DB00001,lepirudin,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB06695,[Drugbank],[GNBR],[DGIdb],NaN,NaN,NaN,"[Compound, DB00001]",Compound
3,DB00001,lepirudin,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB01254,[Drugbank],[GNBR],[DGIdb],NaN,NaN,NaN,"[Compound, DB00001]",Compound
4,DB00001,lepirudin,Compound::DB00001,DRUGBANK::ddi-interactor-in::Compound:Compound,Compound::DB01609,[Drugbank],[GNBR],[DGIdb],NaN,NaN,NaN,"[Compound, DB00001]",Compound
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209572,DB15598,ferric maltol,Compound::DB15598,DRUGBANK::x-atc::Compound:Atc,Atc::B03,[Drugbank],[GNBR],NaN,NaN,NaN,NaN,"[Compound, DB15598]",Compound
1209573,DB15598,ferric maltol,Compound::DB15598,DRUGBANK::x-atc::Compound:Atc,Atc::B,[Drugbank],[GNBR],NaN,NaN,NaN,NaN,"[Compound, DB15598]",Compound
1209574,DB15598,ferric maltol,Compound::DB15598,GNBR::T::Compound:Disease,Disease::MESH:D000740,[Drugbank],[GNBR],NaN,NaN,NaN,NaN,"[Compound, DB15598]",Compound
1209575,DB15598,ferric maltol,Compound::DB15598,GNBR::T::Compound:Disease,Disease::MESH:D015212,[Drugbank],[GNBR],NaN,NaN,NaN,NaN,"[Compound, DB15598]",Compound


In [24]:
sider_drugs["drug_name"] = sider_drugs["drug_name"].str.lower()

In [25]:
drug_sider_id = pd.merge(sider_drugs,drug_entities,on="drug_name")
drug_sider = drug_sider_id[["drug_id","ID"]]
drug_sider = drug_sider.drop_duplicates()

In [26]:
side_effects = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/side_effects.csv")
side_effects = side_effects.drop(["drug","unknown","se1","side_effect"],axis = 1)
side_effects = side_effects.drop_duplicates()

In [27]:
side_effects = pd.merge(drug_sider,side_effects,on= "drug_id")

In [28]:
side_effects

,drug_id,ID,se
0,CID100000143,DB00650,C0002170
1,CID100000143,DB00650,C0003123
2,CID100000143,DB00650,C0009450
3,CID100000143,DB00650,C0009806
4,CID100000143,DB00650,C0011603
...,...,...,...
114726,CID154687131,DB00256,C0549567
114727,CID154687131,DB00256,C0851353
114728,CID154687131,DB00256,C0852911
114729,CID154687131,DB00256,C1257843


In [31]:
import itertools
import math
import numpy as np
import pandas as pd
import os
import shutil

In [32]:
def se_similarity(df) : 
    drug_se_dict = {k : g["se"].tolist() for k,g in df.groupby("ID")}
    sim_scores = []

    for combination in itertools.combinations(side_effects["ID"].unique(),2):
        drug1 = combination[0]
        drug2 = combination[1]

        side_effects1 = drug_se_dict[drug1]
        side_effects2 = drug_se_dict[drug2]

        c = set(side_effects1).intersection(side_effects2)
        u = set(side_effects1).union(side_effects2)

        sim_score = len(c) / float(len(u))
        sim_scores.append([drug1,drug2,sim_score])
    drug_se_df = pd.DataFrame(sim_scores, columns = ["Drug1", "Drug2", "SE-SIM"])
    return drug_se_df

In [33]:
se_sim = se_similarity(side_effects)

In [36]:
se_sim.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drugs-se-sim.csv", index= False)